# Optimization-based ODE Inference Example

## Simple Local Optimization
https://docs.juliadiffeq.org/dev/analysis/parameter_estimation/#Simple-Local-Optimization-1

In [5]:
using OrdinaryDiffEq, DifferentialEquations, DiffEqParamEstim
using JuMP, NLopt
using Plots
gr()

Plots.GRBackend()

In [6]:
function f(du,u,p,t)
  du[1] = dx = p[1]*u[1] - u[1]*u[2]
  du[2] = dy = -3*u[2] + u[1]*u[2]
end

u0 = [1.0;1.0]
tspan = (0.0,10.0)
p = [1.5]
prob = ODEProblem(f,u0,tspan,p)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 10.0)
u0: [1.0, 1.0]

In [7]:
sol = solve(prob,Tsit5())
t = collect(range(0,stop=10,length=200))
using RecursiveArrayTools # for VectorOfArray
randomized = VectorOfArray([(sol(t[i]) + .01randn(2)) for i in 1:length(t)])
data = convert(Array,randomized)

2×200 Array{Float64,2}:
 0.992466  1.03257   1.06868   1.10157   …  0.964115  1.0094   1.03166 
 0.996542  0.897328  0.819717  0.741255     1.11431   1.01208  0.900618

In [8]:
function g(du,u,p,t)
  σ,ρ,β = p
  x,y,z = u
  du[1] = dx = σ*(y-x)
  du[2] = dy = x*(ρ-z) - y
  du[3] = dz = x*y - β*z
end

g (generic function with 1 method)

In [9]:
u0 = [1.0;0.0;0.0]
t = 0.0:0.01:1.0
tspan = (0.0,1.0)
model_ode(p_) = ODEProblem(g, u0, tspan,p_)
solve_model(mp_) = OrdinaryDiffEq.solve(model_ode(mp_), Tsit5(),saveat=0.01)
mock_data = Array(solve_model([10.0,28.0,8/3]))

3×101 Array{Float64,2}:
 1.0  0.917924   0.867919    0.84536     …  -9.4328   -9.4199   -9.3954 
 0.0  0.26634    0.511741    0.744654       -9.36254  -9.23248  -9.09342
 0.0  0.0012639  0.00465545  0.00983587     28.3322   28.4516   28.5537 

In [10]:
loss_objective(mp_, dat) = build_loss_objective(
    model_ode(mp_), Tsit5(), L2Loss(t,dat))

loss_objective (generic function with 1 method)

In [12]:
juobj(args...) = loss_objective(args, mock_data)(args)
jumodel = Model()
JuMP.register(jumodel, :juobj, 3, juobj, autodiff=true)
@variables jumodel begin
    σ,(start=8)
    ρ,(start=25.0)
    β,(start=10/3)
end
@NLobjective(jumodel, Min, juobj(σ, ρ, β))
setsolver(jumodel, NLoptSolver(algorithm=:LD_MMA))

UndefVarError: UndefVarError: setsolver not defined